# Migrating from ConversationBufferMemory

[ConversationBufferMemory](https://api.python.langchain.com/en/latest/memory/langchain.memory.buffer.ConversationBufferMemory.html) was used to keep track of a conversation between a human and an ai asstistant.

The `ConversationBufferMemory` implementation was limited and only worked for:

1. LLMs (i.e., non chat models)
2. Chat Models without native tool calling capability.

We recommenbd that users swap to [persistence using langgraph](https://langchain-ai.github.io/langgraph/how-tos/persistence/).

Some advantages to using LangGraph persistence:

- LangGraph persistence is much more powerful than simple chat memory -- it lets you save and resume complex state at any time for error recovery, human-in-the-loop workflows, time travel interactions, and more.
- LangGraph persistence is built to support multi user, multi thread scenarios and will work well in production settings.
- Both LLMs and chat models are fully supported.
- Users have granular control over how memory is updated.

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain-openai langchain langchain-community

In [2]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

## LLMChain / ConversationChain

### Legacy

Below is example usage of `ConversationBufferMemory` with an `LLMChain` or an equivalent `ConversationChain`.

<details open>

In [3]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

prompt = ChatPromptTemplate([
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{text}"),
])

memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True
)

legacy_chain = LLMChain(
    llm=ChatOpenAI(), prompt=prompt, memory=memory)

legacy_result = legacy_chain.invoke({"text": "my name is bob"})
print(legacy_result)

legacy_result = legacy_chain.invoke({"text": "what was my name"})

/tmp/ipykernel_925844/1840646419.py:23: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  legacy_chain = LLMChain(


{'text': 'Nice to meet you, Bob! How can I assist you today?', 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})]}


Note that `LLMChain` by default returned a `dict` containing both the input and the output from `StrOutputParser`, so to extract the output, you need to access the `"text"` key.

In [4]:
legacy_result["text"]

'Your name is Bob. How can I assist you further, Bob?'

</details>

### LangGraph

<details open>

In [5]:
from langgraph.graph import StateGraph, START, MessagesState
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

model = ChatOpenAI()

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}

# Define the two nodes we will cycle between
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
memory = MemorySaver() # Add in persistence
app = workflow.compile(checkpointer=memory)

# Let's test out application
# The thread_id will be a key that will be used to identify
# this particular conversation.
config = {"configurable": {"thread_id": "2"}}

input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Here, let's confirm that the AI remembers our name!
config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="what was my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hello Bob! How can I assist you today?
================================ Human Message =================================

what was my name?
================================== Ai Message ==================================

Your name is Bob. Is there anything else you would like to know or talk about?


</details>

## Agent Executor

This shows usage of an Agent Executor with an agent constructed using [create_tool_calling_agent](https://api.python.langchain.com/en/latest/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html) function.

The same logic applies to idea applies to other pre-built agents which can be the pre-built [create_react_agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).

### Legacy Usage

<details open>

In [1]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_core.tools import tool
from langchain.memory import ConversationBufferMemory


model = ChatOpenAI(temperature=0)

@tool
def get_user_age(name: str) -> str:
    """Use this tool to find the user's age."""
    # This is a placeholder for the actual implementation
    if 'bob' in name.lower():
        return '42 years old'
    return '41 years old'

tools = [get_user_age]


# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Instantiate memory
memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True
)

# Create an agent
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory)

# Verify that the agent can use tools
print(agent_executor.invoke({"input": "hi! my name is bob what is my age?"}))
print()
# Verify that the agent has access to conversation history.
# The agent should be able to answer that the user's name is bob.
print(agent_executor.invoke({"input": "do you remember my name?"}))

/home/eugene/.pyenv/versions/3.11.4/envs/langgraph_3_11/lib/python3.11/site-packages/langsmith/client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


{'input': 'hi! my name is bob what is my age?', 'chat_history': [HumanMessage(content='hi! my name is bob what is my age?', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Bob! You are 42 years old. If you need any more assistance or information, feel free to ask!', additional_kwargs={}, response_metadata={})], 'output': 'Hello Bob! You are 42 years old. If you need any more assistance or information, feel free to ask!'}

{'input': 'do you remember my name?', 'chat_history': [HumanMessage(content='hi! my name is bob what is my age?', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Bob! You are 42 years old. If you need any more assistance or information, feel free to ask!', additional_kwargs={}, response_metadata={}), HumanMessage(content='do you remember my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, your name is Bob. How can I assist you further, Bob?', additional_kwargs={}, response_metadata={})], 'output':

</details>

### LangGraph

If you want to use a pre-built agent, where you only supply the tools and the prompt, you can follow [this guide](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent-memory/).

<details open>

In [3]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

@tool
def get_user_age(name: str) -> str:
    """Use this tool to find the user's age."""
    # This is a placeholder for the actual implementation
    if 'bob' in name.lower():
        return '42 years old'
    return '41 years old'

memory = MemorySaver()
model = ChatOpenAI()
app = create_react_agent(model, tools=[get_user_age], checkpointer=memory)

# Let's test out application
# The thread_id will be a key that will be used to identify
# this particular conversation.
config = {"configurable": {"thread_id": "2"}}

# Tell the AI that our name is Bob, and ask it to use a tool to confirm
# that it's capable of working like an agent.
input_message = HumanMessage(content="hi! I'm bob. What is my age?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Confirm that the chat bot has access to previous conversation
# and can respond to the user saying that the user's name is Bob.
input_message = HumanMessage(content="do you remember my name?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob. What is my age?
================================== Ai Message ==================================
Tool Calls:
  get_user_age (call_E4SDRy9Hgi5t9Vt5g73Rxjst)
 Call ID: call_E4SDRy9Hgi5t9Vt5g73Rxjst
  Args:
    name: bob
================================= Tool Message =================================
Name: get_user_age

42 years old
================================== Ai Message ==================================

Bob, you are 42 years old.
================================ Human Message =================================

do you remember my name?
================================== Ai Message ==================================

Yes, your name is Bob.


In [4]:
# Now, let's confirm that if we use a different thread
# that the bot will not know what our name is since
# it's a new conversation!
config = {"configurable": {"thread_id": "123456789"}}

input_message = HumanMessage(content="hi! I'm bob. What is my age?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

hi! I'm bob. What is my age?
================================== Ai Message ==================================
Tool Calls:
  get_user_age (call_IkvTlGo0Jgy0JRSU7cpculkl)
 Call ID: call_IkvTlGo0Jgy0JRSU7cpculkl
  Args:
    name: Bob
================================= Tool Message =================================
Name: get_user_age

42 years old
================================== Ai Message ==================================

Hello Bob! You are 42 years old. If you need any more information or assistance, feel free to ask!


</details>

## Next steps

* [LangGraph quickstart tutorial](https://langchain-ai.github.io/langgraph/tutorials/introduction/)
* [How to add persistence ("memory") to your graph](https://langchain-ai.github.io/langgraph/how-tos/persistence/)
* [How to manage conversation history](https://langchain-ai.github.io/langgraph/how-tos/memory/manage-conversation-history/)
* [How to add summary of the conversation history](https://langchain-ai.github.io/langgraph/how-tos/memory/add-summary-conversation-history/)